In [3]:
using Distributions
n = 10  # number of observations
k = 15  # number of markers
varg = 15  # total genetic genetic variance
vare = 1.0  # residual variance
vara = varg/k 
λ = vare/vara
srand(123)
M = sample([0,1,2],(n,k))
mu = 10
ONE = ones(n)
mk_effect = rand(Normal(0.0, sqrt(vara)), k)
y = ONE*mu + M*mk_effect + rand(Normal(0.0, sqrt(vare)), n)
lhs = [ONE'ONE ONE'M
    M'ONE M'M + λ*eye(k)]
rhs = [ONE'y
      M'y]
invlhs = inv(lhs)
sol = invlhs*rhs
GEBV =  M*sol[2:end]

10-element Array{Float64,1}:
 -0.964976
 -1.63135 
 -1.10211 
 -4.84306 
  0.642259
  3.0403  
 -7.43588 
 -1.20203 
 -0.810783
 -0.591994

In [76]:
# using gibbs sampling to solve the MME
mu = mean(y)
alpha = fill(0.01,k)
inter = 50000
burnin = 20000
step = 50
alphastore = fill(0.0, inter, k)
alphastore2 = fill(0.0, int((inter-burnin)/step + 1), k)
srand(123)
for i=1:inter
    # sampling mu
    me = (ONE'y - ONE'M*alpha)[1]/n
    va = vare/n
    mu = me + randn()*sqrt(va)
    # sampling alpha
    for j=1:k
        temp = vec(lhs[j+1, :])
        splice!(temp, j+1)
        temp2 = copy(alpha)
        splice!(temp2, j)
        me = ((M'y)[j] - dot(temp, [mu, temp2]))/lhs[j+1,j+1] 
        va = vare/lhs[j+1,j+1]
        alpha[j] = me + randn()*sqrt(va)
    end
    alphastore[i,:] = alpha   
end

alphastore2 = alphastore[burnin:step:inter, :] 

alpha=mean(alphastore2, 1)
GEBV4=M*alpha'

10x1 Array{Float64,2}:
 -0.949853
 -1.57414 
 -1.0673  
 -4.80255 
  0.660154
  3.03097 
 -7.40893 
 -1.2354  
 -0.786967
 -0.559263

In [4]:
# using gibbs sampling to solve the MME
mu = mean(y)
#alpha = fill(0.01,k)
sol = fill(0.01,k+1)
inter = 50000
burnin = 20000
step = 50
alphastore = fill(0.0, inter, k)
alphastore2 = fill(0.0, int((inter-burnin)/step + 1), k)
srand(123)
for i=1:inter
    # sampling mu and alpha
    for j=1:k+1
        temp = vec(lhs[j, :])
        splice!(temp, j)
        temp2 = copy(sol)
        splice!(temp2, j)
        me = (rhs[j] - dot(temp, temp2))/lhs[j,j] 
        va = vare/lhs[j,j]
        sol[j] = me + randn()*sqrt(va)
    end
    alphastore[i,:] = sol[2:end]   
end

alphastore2 = alphastore[burnin:step:inter, :] 

alpha=mean(alphastore2, 1)
GEBV4=M*alpha'

10x1 Array{Float64,2}:
 -0.949853
 -1.57414 
 -1.0673  
 -4.80255 
  0.660154
  3.03097 
 -7.40893 
 -1.2354  
 -0.786967
 -0.559263

In [74]:
alpha = fill(0.01,k)
[mu,alpha]

16-element Array{Float64,1}:
 11.832
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 
  0.01 

In [64]:
# BayesC0 for the sample example as upper
function sampleEffects!(nMarkers,xArray,xpx,yCorr,α,meanAlpha,vare,varEffects)
    #nObs = 10
    for j=1:nMarkers
        rhs      = dot(xArray[:,j],yCorr) + xpx[j]*α[j]   
        lhs      = xpx[j] + vare/varEffects
        invLhs   = 1.0/lhs
        mean     = invLhs*rhs
        oldAlpha = α[j] 
        α[j] = mean + randn()*sqrt(invLhs*vare)
        BLAS.axpy!(oldAlpha-α[j],xArray[:,j],yCorr)  
    end
    nothing
end
srand(123)
meanMu    = 0
meanAlpha = zeros(k)
varEffects = 1.0
mu = mean(y)
yCorr = y - mu
alpha = fill(0.01,k)
nObs = 10
mpm=diag(M'M)
vare=1.0
inter = 50000
burnin = 20000
step = 20
alphastore = fill(0.0, inter, k)
alphastore2 = fill(0.0, int((inter-burnin)/step + 1), k)
for i=1:inter        
        # sample intercept
        yCorr  = yCorr+mu
        rhs    = sum(yCorr)                   
        invLhs = 1.0/(nObs)               
        mean   = rhs*invLhs   
        mu     = mean + randn()*sqrt(invLhs*vare)
        yCorr  = yCorr - mu                    
        meanMu = meanMu + (mu - meanMu)/i
        
        # sample effects
    sampleEffects!(k,M,mpm,yCorr,alpha,meanAlpha,vare,varEffects)
    alphastore[i,:] = alpha        
end
alphastore2 = alphastore[burnin:step:inter, :] 

alpha=mean(alphastore2, 1)
GEBV3 = M*alpha'

10x1 Array{Float64,2}:
 -0.809879
 -1.49086 
 -1.03103 
 -4.70261 
  0.717804
  3.04628 
 -7.29235 
 -1.15779 
 -0.697985
 -0.523534